In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")

/Users/familiasofi/Documents/Repositorios_GitHub/cv_recommendations/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%time
pipe("Uber", top_k=None)

CPU times: user 145 ms, sys: 14.4 ms, total: 159 ms
Wall time: 79.2 ms


[{'label': 'neutral', 'score': 0.8366278409957886},
 {'label': 'positive', 'score': 0.11137551069259644},
 {'label': 'negative', 'score': 0.05199667438864708}]

In [3]:
import pandas as pd

# Crear el dataset de entrenamiento con datos dummy
data = {
    'text': [
        'He comprado un nuevo teléfono móvil',      # Compra
        'Vendí mi coche viejo por un buen precio',  # Venta
        'Estoy buscando ofertas para comprar una laptop',  # Compra
        'Acabo de vender mi consola de videojuegos',  # Venta
        'Estoy pensando en comprar una bicicleta eléctrica',  # Compra
        'La tienda vende productos electrónicos a buen precio',  # Venta
        'Hoy compré varios libros para leer',  # Compra
        'Vendo muebles antiguos en buenas condiciones',  # Venta
        'Voy a comprar una casa en la playa',  # Compra
        'He vendido mi colección de relojes antiguos',  # Venta
        'Estoy comprando entradas para un concierto',  # Compra
        'Vendo una cámara fotográfica que ya no uso'  # Venta
    ],
    'label': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]  # 0 = Compra, 1 = Venta
}

# Convertir el diccionario a un DataFrame
df = pd.DataFrame(data)

In [4]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import time

# Convertir el dataframe a un dataset de HuggingFace
train_dataset = Dataset.from_pandas(df)

# Cargar el tokenizer y el modelo preentrenado
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Tokenizar el texto
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de salida
    num_train_epochs=1,              # Número de épocas
    per_device_train_batch_size=16,   # Tamaño del batch por dispositivo
    logging_dir='./logs',            # Directorio de logs
    logging_steps=10,
)

# Definir el Trainer
trainer = Trainer(
    model=model,                         # El modelo a fine-tunear
    args=training_args,                  # Argumentos de entrenamiento
    train_dataset=train_dataset,         # Dataset de entrenamiento
)

# Medir el tiempo de entrenamiento
start_time = time.time()

# Fine-tunear el modelo
trainer.train()

end_time = time.time()

# Guardar el tiempo de fine-tuning
training_time = end_time - start_time

print(f"Tiempo de fine-tuning: {training_time:.2f} segundos")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1/1 [00:14<00:00, 14.30s/it]

{'train_runtime': 14.2933, 'train_samples_per_second': 0.84, 'train_steps_per_second': 0.07, 'train_loss': 0.7041720747947693, 'epoch': 1.0}
Tiempo de fine-tuning: 14.48 segundos


In [12]:
# Realizar una predicción con el modelo fine-tuneado
model.eval()  # Ponemos el modelo en modo evaluación

# Ejemplo de texto para predecir (puedes cambiarlo por cualquier texto nuevo)
text_example = "Estoy comprando un televisor nuevo"

# Tokenizar el ejemplo de texto
inputs = tokenizer(text_example, return_tensors="pt", padding=True, truncation=True)

# Medir el tiempo de predicción
start_pred_time = time.time()

# Realizar la predicción
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()

end_pred_time = time.time()

# Guardar el tiempo de predicción
inference_time = end_pred_time - start_pred_time
print(f"Tiempo de predicción: {inference_time:.2f} segundos")

# Mostrar la predicción
print(f"Predicción: {'Compra' if prediction == 0 else 'Venta'}")

Tiempo de predicción: 0.05 segundos
Predicción: Compra


In [22]:
model.save_pretrained("distilbert_finetunned")

# Guardado del modelo para una inferencia con ONNX:

In [ ]:
!pip install optimum
!pip install onnxruntime
!pip install onnx
!pip install cupy

In [28]:
import torch
from transformers import DistilBertForSequenceClassification, AutoTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

# Define the path to your fine-tuned model
finetuned_model_path = "microsoft/deberta-v3-small"  # Your fine-tuned model path
original_model_path = "microsoft/deberta-v3-small"
save_directory = "onnx_model/"

# Load a model from transformers and export it to ONNX
ort_model = ORTModelForSequenceClassification.from_pretrained(finetuned_model_path, export=True)
tokenizer = AutoTokenizer.from_pretrained(original_model_path)

# Save the onnx model and tokenizer
ort_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

KeyError: 'deberta-v2 is not supported yet with the onnx backend. Only [] are supported. If you want to support onnx please propose a PR or open up an issue.'

In [6]:
from transformers import AutoTokenizer
from optimum.onnxruntime import (
    AutoQuantizationConfig,
    ORTModelForSequenceClassification,
    ORTQuantizer
)
from optimum.pipelines import pipeline

# Load the tokenizer and export the model to the ONNX format
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
save_dir = "distilbert_quantized"

model = ORTModelForSequenceClassification.from_pretrained(model_id, export=True)

# Load the quantization configuration detailing the quantization we wish to apply
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=True)
quantizer = ORTQuantizer.from_pretrained(model)

# Apply dynamic quantization and save the resulting model
quantizer.quantize(save_dir=save_dir, quantization_config=qconfig)
# Load the quantized model from a local repository
model = ORTModelForSequenceClassification.from_pretrained(save_dir)

# Create the transformers pipeline
onnx_clx = pipeline("text-classification", model=model, accelerator="ort")
text = "I like the new ORT pipeline"
pred = onnx_clx(text)
print(pred)
# [{'label': 'POSITIVE', 'score': 0.9974810481071472}]


[{'label': 'POSITIVE', 'score': 0.9978128671646118}]


In [7]:
%%time
text = "I like the new ORT pipeline"
pred = onnx_clx(text)

CPU times: user 33.3 ms, sys: 3.56 ms, total: 36.9 ms
Wall time: 12 ms


In [31]:
!pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 3.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [32]:
from sentence_transformers import SentenceTransformer

model_id = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_id, backend="onnx")

sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = model.encode(sentences)

Multiple ONNX files found in 'sentence-transformers/all-MiniLM-L6-v2': ['onnx/model.onnx', 'onnx/model_O1.onnx', 'onnx/model_O2.onnx', 'onnx/model_O3.onnx', 'onnx/model_O4.onnx', 'onnx/model_qint8_arm64.onnx', 'onnx/model_qint8_avx512.onnx', 'onnx/model_qint8_avx512_vnni.onnx', 'onnx/model_quint8_avx2.onnx'], defaulting to 'onnx/model.onnx'. Please specify the desired file name via `model_kwargs={"file_name": "<file_name>"}`.
2024-11-09 16:58:34.291092 [W:onnxruntime:, helper.cc:82 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight, shape: {30522,384}
2024-11-09 16:58:34.292814 [W:onnxruntime:, coreml_execution_provider.cc:107 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 55 number of nodes in the graph: 418 number of nodes supported by CoreML: 278
Context leak detected, CoreAnalytics returned false
Context leak detected, CoreAnalytics returned false
2024-11-09 16:58:35.787828 [W:onnxru

Fail: [ONNXRuntimeError] : 1 : FAIL : Non-zero status code returned while running CoreML_11068300627432712736_1 node. Name:'CoreMLExecutionProvider_CoreML_11068300627432712736_1_1' Status Message: Error executing model: Unable to compute the prediction using a neural network model. It can be an invalid input data or broken/unsupported model (error code: -1).

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification

# Load the model from the hub and export it to the ONNX format
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
model = ORTModelForSequenceClassification.from_pretrained(model_id, export=True)

In [33]:
from optimum.onnxruntime import ORTModelForSequenceClassification

# Load the model from the hub and export it to the ONNX format
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
model = ORTModelForSequenceClassification.from_pretrained("Notebooks/distilbert_finetunned", export=True)



RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-672fdb6f-3757578a7c05bbb900d82f7d;b8e08fa2-f47c-4626-ab35-1c7231df389d)

Repository Not Found for url: https://huggingface.co/api/models/Notebooks/distilbert_finetunned/tree/main?recursive=True&expand=False.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [3]:
from optimum.pipelines import pipeline

classification = pipeline(task="text-classification", accelerator="ort")

In [5]:
%%time
classification("Hola")

CPU times: user 46.3 ms, sys: 3.18 ms, total: 49.5 ms
Wall time: 30.3 ms


[{'label': 'POSITIVE', 'score': 0.7184391021728516}]

In [40]:
pipeline??

Signature:
pipeline(
    task: str = None,
    model: Optional[Any] = None,
    tokenizer: Union[str, transformers.tokenization_utils.PreTrainedTokenizer, NoneType] = None,
    feature_extractor: Union[str, ForwardRef('SequenceFeatureExtractor'), NoneType] = None,
    use_fast: bool = True,
    token: Union[bool, str, NoneType] = None,
    accelerator: Optional[str] = 'ort',
    revision: Optional[str] = None,
    trust_remote_code: Optional[bool] = None,
    *model_kwargs,
    **kwargs,
) -> transformers.pipelines.base.Pipeline
Docstring: <no docstring>
Source:   
def pipeline(
    task: str = None,
    model: Optional[Any] = None,
    tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    feature_extractor: Optional[Union[str, PreTrainedFeatureExtractor]] = None,
    use_fast: bool = True,
    token: Optional[Union[str, bool]] = None,
    accelerator: Optional[str] = "ort",
    revision: Optional[str] = None,
    trust_remote_code: Optional[bool] = None,
    *model_kwargs,
